In [ ]:
import csv
import sys
import time
sys.path.append("../libs/")
import tactusloaderLIB
import OWEmailSorterLIB
import markduplicatesLIB
import removemarkedtextLIB
import LIWCLIB

In [ ]:
DIRECTORY = "/home/erikt/projects/e-mental-health/usb/tmp/20190917"

In [ ]:
DEFAULTOUTFILE="out.csv"
FIELDNAMEDATE = "date"

def addZero(string):
    while len(string) < 2: string = "0"+string
    return(string)

def time2str(timeObj):
    date = str(timeObj.tm_year)+"-"+addZero(str(timeObj.tm_mon))+"-"+addZero(str(timeObj.tm_mday))
    time = addZero(str(timeObj.tm_hour))+":"+addZero(str(timeObj.tm_min))+":"+addZero(str(timeObj.tm_sec))
    return(date+" "+time)

def floatPrecision5(number):
    if type(number) != type(0.5): return(number)
    else: return(float("{0:.5f}".format(number)))

def saveResults(allLiwcResults,fileName=DEFAULTOUTFILE):
    if len(allLiwcResults) > 0:
        fieldNames = [x.name for x in allLiwcResults[0].domain.metas]
        fieldNames += [x.name for x in allLiwcResults[0].domain.variables]
        outFile = open(fileName,"w")
        with outFile as csvFile:
            csvwriter = csv.DictWriter(csvFile,fieldnames=fieldNames)
            csvwriter.writeheader()
            for liwcResults in allLiwcResults:
                for liwcResultsRow in liwcResults:
                    row = {}
                    for fieldName in fieldNames:
                        if fieldName != FIELDNAMEDATE: row[fieldName] = floatPrecision5(liwcResultsRow[fieldName].value)
                        else: row[fieldName] = time2str(time.localtime(liwcResultsRow[fieldName].value))
                    csvwriter.writerow(row)
        outFile.close()

In [ ]:
allLiwcResults = []
for patientId in range(1,9):
    fileName = tactusloaderLIB.makeFileName(str(patientId))
    mails = tactusloaderLIB.processFile(DIRECTORY,fileName)
    sortedMails = OWEmailSorterLIB.filterEmails(mails,filter_asc=True)
    markedMails = markduplicatesLIB.processCorpus(sortedMails)
    strippedMails = removemarkedtextLIB.processCorpus(markedMails)
    liwcResults = LIWCLIB.processCorpus(strippedMails)
    allLiwcResults.append(liwcResults)
saveResults(allLiwcResults)